# Defining the model

## Downloading the pre-trained model

We import the model without the top layer, with the pretrained imagenet weights and an input shape of (299, 299, 3).

In [ ]:
from keras.applications.xception import Xception
base_model = Xception(
        include_top=False,
        weights='imagenet',
        input_shape=(299, 299, 3))

## Freeze the layers

In [ ]:
base_model.trainable = False

## Add new layers

This layers will be the ones that we will train at the beginning. This **new** part of the network will learn to identify the new classes that we want to classify.

In [ ]:
from keras.applications.xception import Xception, preprocess_input
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.losses import categorical_crossentropy
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D
from keras.models import Model
from keras import Input
from keras.regularizers import l2

inputs = Input(shape=(299, 299, 3))
x = base_model(inputs, training=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu',  kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu',  kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)
predictions = Dense(6, activation='softmax')(x)
model = Model(inputs=base_model.inputs, outputs=predictions)

## Compile the model

Here we 'save' the changes we have made to the model so we can tain it on its new form.

In [ ]:
model.compile(
    loss=categorical_crossentropy,
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

# Data augmentation

Sometimes our data won't be enough to train a good model. In this case, we can use data augmentation to generate new images from the ones we already have. This will help the model to generalize better and avoid overfitting.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import preprocess_input
## In case a custom preprocessing function is to be used
# train_datagen = ImageDataGenerator(
#     rescale=1./225,
#     rotation_range=25,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )
train_datagen = ImageDataGenerator(dtype='float32',preprocessing_function=preprocess_input)
validation_datagen = ImageDataGenerator(rescale=1./255)

In this case we will use the preprocessing function that comes with the model. This function will normalize the images and will also do some data augmentation. In addition, we don't want to augment the validation data, so we will use two different preprocessing functions.

## Generators

In [ ]:
train_dir = 'your/path/to/train'
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=10,
    class_mode='categorical'
)

validation_dir = 'your/path/to/validation'
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(299, 299),
    batch_size=10,
    class_mode='categorical'
)

# Training the model

## Callbacks

Callbacks are functions that are called at certain points during the training process. We will use them to save the model when it achieves the best validation accuracy and to stop the training if the validation accuracy doesn't improve after a certain number of epochs.

In [ ]:
# Create a learning rate scheduler callback
from tensorflow import keras
from keras.callbacks import LearningRateScheduler, EarlyStopping

exp_decay = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.5)
lr_scheduler = LearningRateScheduler(exp_decay)

# Create an early stopping callback
patience = 2
early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True, verbose=1)

## Training

The training process can be computationally expensive, so we will use the GPU to speed up the process. The number of epochs is set to a really high number that we won't really achieve since EarlyStopping will stop the training when the validation loss doesn't improve after a certain number of epochs.

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size,
    epochs=10000,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples/validation_generator.batch_size,
    callbacks=[lr_scheduler, early_stopping]
)

## Saving

This cell is used as a security measure. If the fine tunning process is interrupted, we can save the model so we don't have to start the training from the beginning.

In [ ]:
model.save('your/path/to/model.h5', save_format='h5')

# Fine tunning

In this section we will be unfreezing some parts of the model and train them with a really low learning rate. This will help the model to learn more specific features and improve its accuracy.

## Loading the model

In [ ]:
from tensorflow import keras
model = keras.models.load_model('your/path/to/model.h5')

## First iteration

### Unfreezing parts of the model

First of all, we will unfreeze the 18 last layers of the model. This layers will be trained with a really low learning rate and learn more specific features.

In [ ]:
for layer in model.layers[120:]:
  layer.trainable = True

### Training

In [ ]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
model.compile(
    loss=categorical_crossentropy,
    optimizer=Adam(learning_rate=0.000001),
    metrics=['accuracy']
)

In [ ]:
final_history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples/validation_generator.batch_size,
    callbacks=[early_stopping]
)

In [ ]:
model.save('your/path/to/model_phase1.h5', save_format='h5')

## Rest of the iterations

Here we will be repeating the previous section on a while loop for code cleaness, but the process is the same: unfreezing layers 20 by 20 and training them with a really low learning rate.

In [ ]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

n = 3

while n > 0:
  for layer in model.layers[n*20:]:
    layer.trainable = True

  model.compile(
      loss=categorical_crossentropy,
      optimizer=Adam(learning_rate=0.0001),
      metrics=['accuracy']
  )
  final_history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size,
      epochs=25,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      callbacks=[early_stopping]
  )
  model.save(f"your/path/to/model_phase{5-n}.h5", save_format='h5')
  print(f"Model saved on your/path/to/model_phase{5-n}.h5")
  n = n-1

Lastly, we unfreeze the whole model and train it.

In [ ]:
for layer in model.layers[:]:
    layer.trainable = True

model.compile(
    loss=categorical_crossentropy,
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)
final_history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples/validation_generator.batch_size,
    callbacks=[early_stopping]
)
model.save(f"your/path/to/model_phase_final.h5", save_format='h5')

Now you have your own model you can test on your own test set via the `model.predict()` or `model.evaluate()` functions.